In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('dataset.csv', names=['Age','Gender','TB','DB','Alkphos','Sgpt','Sgot','TP','ALG','A/G','Dataset'])
# data = pd.read_csv("dataset.csv")

In [3]:
data

,Age,Gender,TB,DB,Alkphos,Sgpt,Sgot,TP,ALG,A/G,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Age      583 non-null    int64  
 1   Gender   583 non-null    object 
 2   TB       583 non-null    float64
 3   DB       583 non-null    float64
 4   Alkphos  583 non-null    int64  
 5   Sgpt     583 non-null    int64  
 6   Sgot     583 non-null    int64  
 7   TP       583 non-null    float64
 8   ALG      583 non-null    float64
 9   A/G      579 non-null    float64
 10  Dataset  583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


# Preprocess
## Missing Values

In [5]:
data.isna().sum()

Age        0
Gender     0
TB         0
DB         0
Alkphos    0
Sgpt       0
Sgot       0
TP         0
ALG        0
A/G        4
Dataset    0
dtype: int64

In [6]:
data['A/G'] = data['A/G'].fillna(data['A/G'].mean())

# Encoding

In [7]:
def binary_encode(df, column, positive_value):
    df = df.copy()
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

In [8]:
data = binary_encode(data, 'Gender', 'Male')

# Lets change the label to 0,1 instead of 1,2

In [9]:
data = binary_encode(data, 'Dataset', 1)

# Splitting and Scalling

In [10]:
data

,Age,Gender,TB,DB,Alkphos,Sgpt,Sgot,TP,ALG,A/G,Dataset
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,0
579,40,1,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,1,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,1,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [11]:
y = data['Dataset']
X = data.drop('Dataset', axis=1)

In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# Training

In [14]:
X.shape

(583, 10)

In [15]:
y.sum() / len(y)

0.7135506003430532

In [16]:
inputs = tf.keras.Input(shape=(10,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

batch_size = 64
epochs = 100
history = model.fit(
    X_train, y_train, validation_split=0.2,
    batch_size=batch_size,
    epochs = epochs,
    verbose=0
)

#  Results

In [19]:
fig = px.line(
    history.history,
    y=['loss','val_loss'],
    labels={'index':"Epoch", 'value':"Loss" },
    title = "Training and Validation Loss"
)

fig.show()

NameError: name 'px' is not defined

In [20]:
np.argmin(history.history['val_loss'])+1

2

In [21]:
model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 60ms/step - loss: 0.4685 - accuracy: 0.7350 - auc: 0.7561


[0.4685003161430359, 0.7350427508354187, 0.7560736536979675]